In [1]:
"""環境"""
# .lab

"""更新履歴"""
# 最終更新日：2025/05/21
# 2025/05/21 shutilを使って、cachedirを洗浄することにしました
# 2025/05/15 simple_progress_barおよびdata(dict)を使い始めました。
# 2025/02/25 os.getcwd()を採用し、__file__を定義する必要をなくしました。

"""モジュール読み込み"""
# ファイル操作等
import sys
import os
# from datetime import datetime
from pprint import pprint
import logging
# import pickle
# import struct
from tqdm import tqdm
import h5py
# import threading
import json

# tkinter
from tkinter import filedialog, messagebox, Tk

# データ分析ツール
import pandas as pd
import numpy as np
import scipy as sp
# import math
# from sklearn.linear_model import LinearRegression

# グラフ等作成用
import matplotlib
import matplotlib.pyplot as plt         # 図の作成用
from PIL import Image as im
# import cv2
from IPython.display import display, HTML, clear_output, update_display, Image

# 自作モジュール
sys.path.append(r"C:\Users\okaza\pythonenv")
from modules.Mytools.Tools import print_fileinfo, h5_tree, dict_tree, simple_progress_bar, clean_cache_except_logfiles, get_total_size
import modules.Mytools.Settings
import modules.fitXRD as fx
from modules.peakfit import peakfit, pseudoVoigt

"""ログ管理ツール作成"""
# chche directoryの設定
cachedir = os.path.abspath(os.getcwd() + "/.cache")
clean_cache_except_logfiles(cachedir)
os.makedirs(cachedir, exist_ok=True)

# loggerの作成
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = "%(levelname)-9s  %(asctime)s [%(filename)s:%(lineno)d] %(message)s"

# Streamハンドラクラスを作成
sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)
sh.setFormatter(logging.Formatter(format))
logger.addHandler(sh)

# Fileハンドラクラスをインスタンス化
logfile = cachedir + ""
fh = logging.FileHandler(filename=cachedir + "/notebook.log", encoding="utf-8")
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter(format))
logger.addHandler(fh)
logger.debug("[Activate workspace]: " + os.getcwd())
logger.debug("[Set cache]: "+ os.path.abspath(cachedir))
logger.debug("[Activate log]: " + os.path.abspath(cachedir + "/notebook.log"))

# globalなデータを作成
data = dict()
logger.debug("[Create global variable]: data")

DEBUG      2025-05-22 21:09:27,637 [388102471.py:71] [Activate workspace]: c:\Users\okaza\pythonenv\lf\count_temperature
DEBUG      2025-05-22 21:09:27,638 [388102471.py:72] [Set cache]: c:\Users\okaza\pythonenv\lf\count_temperature\.cache
DEBUG      2025-05-22 21:09:27,640 [388102471.py:73] [Activate log]: c:\Users\okaza\pythonenv\lf\count_temperature\.cache\notebook.log
DEBUG      2025-05-22 21:09:27,643 [388102471.py:77] [Create global variable]: data


## 目的

2色法で計算した温度を集計します。

## 1. データ可視化

In [2]:
import matplotlib.animation as anim

温度データを読み込みます。

In [11]:
def set_hdffile():

    # hdffile
    hdffile = r"C:\Users\okaza\pythonenv\lf\twocolor\.cache\tempraw.hdf"

    # hdfファイルを読み込む
    with h5py.File(hdffile, mode = "r") as f:
        h5_tree(f, lim = 1)
        pass
    
    # ファイル情報を表示
    print_fileinfo(hdffile)

    # データ格納
    key = sys._getframe().f_code.co_name
    data[key] = dict()
    data[key]["hdffile"] = hdffile
    logger.info("[Add variables]: {} (data = {:.2f} MB)".format(key, float(get_total_size(data))/1024/1024))
    dict_tree(data)
    
    return
set_hdffile()
del set_hdffile

INFO       2025-05-22 21:27:57,514 [4135265458.py:18] [Add variables]: set_hdffile (data = 0.00 MB)


<HDF5 file "tempraw.hdf" (mode r)>
├── frame = 0
├── frame = 1
├── frame = 10
├── frame = 100
├── frame = 101
├── frame = 102
├── frame = 103
├── frame = 104
├── frame = 105
├── frame = 106
├── frame = 107
├── frame = 108
├── frame = 109
├── frame = 11
├── frame = 110
├── frame = 111
├── frame = 112
├── frame = 113
├── frame = 114
├── frame = 115
├── frame = 116
├── frame = 117
├── frame = 118
├── frame = 119
├── frame = 12
├── frame = 120
├── frame = 121
├── frame = 122
├── frame = 123
├── frame = 124
├── frame = 125
├── frame = 126
├── frame = 127
├── frame = 128
├── frame = 129
├── frame = 13
├── frame = 130
├── frame = 131
├── frame = 132
├── frame = 133
├── frame = 134
├── frame = 135
├── frame = 136
├── frame = 137
├── frame = 138
├── frame = 139
├── frame = 14
├── frame = 140
├── frame = 141
├── frame = 142
├── frame = 143
├── frame = 144
├── frame = 145
├── frame = 146
├── frame = 147
├── frame = 148
├── frame = 149
├── frame = 15
├── frame = 150
├── frame = 151
├── frame = 152

計算結果をアニメで出力します

In [14]:
def anim_imshow():

    # 温度範囲
    trange = (1000, 2500)

    # セル番号（入力しなかった場合は平均値が出力されます）
    cell = 3

    # wavelength_window = ((600, 650), (800, 900))

    if True: # Main

        # 変数読み込み
        with h5py.File(data["set_hdffile"]["hdffile"], mode = "r") as f:
            n_frame = len(f.keys()) - 1
            w = np.array(f["wavelength [nm]"][()]) # type: ignore

        # マスク等を設定
        mask = ~np.tril(np.ones([w.shape[0]]*2)).astype(np.bool_)
        base = np.zeros([w.shape[0]]*2)

        # figを定義
        fig, ax = plt.subplots()

        # アニメを描写
        ims = []
        with h5py.File(data["set_hdffile"]["hdffile"], mode = "r") as f:
            for i in (range(n_frame)):

                # 型をコピー
                new = base.copy()

                # データを読み込み
                val = []
                for j in f["frame = {}".format(i)].keys(): # type: ignore
                    val.append(np.array(f["frame = {}".format(i)][j][()])) # type: ignore
                
                # 成形
                try:
                    cell # type: ignore
                except:
                    val = np.mean(np.stack(val, axis = 0), axis = 0)
                else:
                    val = val[cell]
                new[mask] = val
                new = new.T
                new[mask] = val
                extent = (w[0], w[-1], w[-1], w[0])
                
                # imshow
                cmap = ax.imshow(
                    new,
                    vmax = trange[1],
                    vmin = trange[0],
                    extent = extent,
                    alpha = 1,
                )

                # text
                tex = ax.text(0,1.01,"frame number = {:>4}".format(i),fontsize = 12, transform = ax.transAxes)

                # データを追加
                ims.append([cmap, tex])

                # プログレスバー
                simple_progress_bar(i+1, n_frame)

        plt.colorbar(mappable=cmap).set_label("Temperature [K]", fontsize = 10)
        ax.set_xlabel("Wavelength [m]")
        ax.set_ylabel("Wavelength [m]")

        #. Save
        logger.debug("[Save]: Animation")
        ani = anim.ArtistAnimation(fig, ims, interval = 300)
        filename = cachedir + "/anim_imshow.mp4"
        ani.save(filename, writer = "ffmpeg", fps = 30, dpi = 300)
        logger.info("[Save]: " + os.path.abspath(filename))

        plt.close()
    
    return
anim_imshow()
del anim_imshow

Progress: [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■-]  99% (173/175) 

DEBUG      2025-05-22 21:36:31,992 [2669950054.py:73] [Save]: Animation


Progress: [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100% (175/175) 


INFO       2025-05-22 21:37:51,532 [2669950054.py:77] [Save]: c:\Users\okaza\pythonenv\lf\count_temperature\.cache\anim_imshow.mp4
